In [ ]:
import sys, os

sys.path.append("/cr/users/filip/bin/")

# from utils.Auger.SD import Monitoring

from utils.binaries import *
from utils.plotting import *

In [ ]:
from scipy.stats import norm

# see Station.h in Framework/SDetector
ADC_bin_edges = np.array([2 * k for k in range(100)] + [200 + 8 * k for k in range(51)])
ADC = 0.5 * (ADC_bin_edges[1:] + ADC_bin_edges[:-1])
ADC_width = np.diff(ADC_bin_edges)


def make_histo(counts):

    histo = []
    skip_to_peak = 5

    for i, count in enumerate(counts[skip_to_peak:100]):
        idx = i + skip_to_peak
        width = ADC_width[idx]
        wiggle = np.random.uniform(-0.5 * width, 0.5 * width, int(count))
        array = np.array([ADC[idx] for _ in range(int(count))])
        histo += [w + a for w, a in zip(wiggle, array)]

    return histo


def gauss(x, A, mu, sigma):
    return A * np.exp(((x - mu) / (np.sqrt(2) * sigma)) ** 2)

In [ ]:
def create_data():
    container = []
    for month in [
        "2021_12",
        "2022_03",
        "2022_06",
        "2022_09",
        "2022_12",
        "2023_03",
        "2023_06",
        "2023_09",
        "2023_12",
    ]:
        # for month in ['dec2021', 'dec2022', 'dec2023']:
        # for month in ['jun2022', 'jun2023']:
        # for month in ['dec2023']:
        # for month in []

        print(f"going through month: {month}")

        try:
            data = np.loadtxt(f"/cr/users/filip/Data/SDMonitHistos/out_{month}.txt")
        except FileNotFoundError:
            print(f"{month} not in data yet, please create it Ü ...")
            continue

        for i, row in enumerate(data):
            print(f"event {i+1}/{len(data)}", end="\r")
            _id, time, *histo = row
            histogram = make_histo(histo)
            mip_peak, std = norm.fit(histogram)

            rate_above_cut = sum([h > 1.75 * mip_peak for h in histogram]) / 61
            ratio = rate_above_cut / mip_peak
            container.append(
                [str(int(_id)), month, time, mip_peak, rate_above_cut, ratio, histo]
            )

            # break
        # break

    df = pd.DataFrame(container, columns=("id", "month", "time", "histo"))
    df.to_csv(f"uub_randoms_histos_{month}.csv")

    return df


if os.path.isfile(
    f"/cr/tempdata01/filip/SSDCalib/BootstrapHistos/uub_randoms_histos.csv"
):
    df = pd.read_csv(
        "/cr/tempdata01/filip/SSDCalib/BootstrapHistos/uub_randoms_histos.csv",
        index_col=0,
    )
    df["id"] = df["id"].astype(str)
else:
    df = create_data()

In [ ]:
df.head()

In [ ]:
so.jointplot(df, x="mip_peak", y="rate_above_cut", kind="scatter", hue="id")

In [ ]:
high_rate = df.loc[df.rate_above_cut == df.rate_above_cut.max()]
low_rate = df.loc[df.rate_above_cut == df.rate_above_cut.min()]
average_event = df.loc[5]

d = lambda x: np.array([float(a) for a in x[1:-1].split(",")])

plt.plot(ADC, d(average_event.histo) / 61, label='"average rate"')
plt.axvline(average_event.mip_peak, c="k")
plt.plot(ADC, d(high_rate.histo.values[0]) / 61, label='"high rate"')
plt.axvline(high_rate.mip_peak.values[0], c="red", ls="--")
plt.plot(ADC, d(low_rate.histo.values[0]) / 61, label='"low rate"')
plt.axvline(low_rate.mip_peak.values[0], c="blue", ls=":")
plt.legend()

plt.yscale("log")

In [ ]:
dBins = 0.5 * (ADC[1:] + ADC[:-1])
ddBins = 0.5 * (dBins[1:] + dBins[:-1])

Y = d(average_event.histo)
Y /= np.max(Y)
dY = np.diff(Y)
dY /= np.max(dY)
ddY = np.diff(dY)
ddY /= np.max(ddY)

plt.plot(ADC, Y, label="histo")
plt.plot(dBins, dY, label="histo'")
plt.plot(ddBins, ddY, label="histo''")

plt.xlim(0, 120)
plt.legend()

In [ ]:
station_container = []

for station in np.unique(df.id.values):
    selected_rows = df.loc[df["id"] == station]
    x, y = selected_rows["mip_peak"], selected_rows["rate_above_cut"]

    try:
        fit, residuals, _, _, _ = np.polyfit(x, y, 1, full=True)
    except SystemError:
        continue

    chi_sqr = residuals
    station_container.append([station, *fit, chi_sqr])

df_station = pd.DataFrame(
    station_container, columns=("id", "slope", "intercept", "chi_sqr")
)
so.scatterplot(df_station, x="intercept", y="slope")

In [ ]:
import datetime

gps_unix_offset = 315964800

plt.scatter(
    [datetime.datetime.fromtimestamp(int(x) + gps_unix_offset) for x in df.time.values],
    df.mip_peak.values,
    alpha=0.05,
)

print(
    f"Peak to peak = {(df.mip_peak.max() - df.mip_peak.min()) / df.mip_peak.mean() * 100 :.2f}%"
)
print(f"std. dev. = {df.mip_peak.std()/df.mip_peak.mean() * 100 :.2f}%")

In [ ]:
x_data = [datetime.datetime.fromtimestamp(x) for x in df.time.values % 24 * 3600]
plot.box_series(x_data, df.mip_peak.values, fmt="%H:%M", bins=100)

plt.xlabel("time of day")
plt.ylabel("MIP peak / ADC")

# Bootstrapping events to create continuous histos

In [ ]:
with open("/cr/tempdata01/filip/SSDCalib/BootstrapHistos/histos.csv", "w") as file:
    for i, (mip, counts) in enumerate(zip(df.mip_peak, df.histo)):
        print(i + 1, "/", len(df), end="\r")
        counts = [int(float(a)) for a in counts[1:-1].split(",")]
        histogram = make_histo(counts)

        file.write(f"{mip:.3f} {' '.join([str(np.round(_, 3)) for _ in histogram])}\n")